# Building AI Agents with LlamaIndex

Agents are autonomous AI systems that can use tools, make decisions, and accomplish complex tasks. This notebook covers building powerful agents with LlamaIndex.

## Learning Objectives

By the end of this notebook, you will:
1. Understand agent architecture and reasoning patterns
2. Build agents with custom tools
3. Implement ReAct agents for complex reasoning
4. Create multi-tool agents
5. Handle agent memory and state

---

## What are Agents?

**Agents** go beyond simple RAG by:
- **Reasoning** about how to accomplish tasks
- **Selecting** appropriate tools for each step
- **Executing** multi-step plans
- **Iterating** based on results

### Agent vs Query Engine

| Query Engine | Agent |
|--------------|-------|
| Single retrieval + generation | Multiple steps with reasoning |
| Fixed pipeline | Dynamic tool selection |
| One data source | Multiple tools and sources |
| Deterministic | Adaptive |

In [ ]:
# Setup
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
)
from llama_index.core.tools import FunctionTool, QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Configure - use a capable model for agents
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

print("✓ Setup complete!")

## 1. Simple Function Tools

Tools are functions that agents can call. Let's start with simple custom tools:

In [ ]:
# Define simple tool functions
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and return the result."""
    return a * b

def add(a: float, b: float) -> float:
    """Add two numbers and return the result."""
    return a + b

def subtract(a: float, b: float) -> float:
    """Subtract b from a and return the result."""
    return a - b

def divide(a: float, b: float) -> float:
    """Divide a by b and return the result."""
    if b == 0:
        return "Error: Cannot divide by zero"
    return a / b

# Convert to LlamaIndex tools
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)
divide_tool = FunctionTool.from_defaults(fn=divide)

print("✓ Calculator tools created!")
print(f"\nMultiply tool metadata:")
print(f"  Name: {multiply_tool.metadata.name}")
print(f"  Description: {multiply_tool.metadata.description}")

In [ ]:
# Create a ReAct agent with calculator tools
calculator_agent = ReActAgent.from_tools(
    tools=[multiply_tool, add_tool, subtract_tool, divide_tool],
    llm=Settings.llm,
    verbose=True,  # Show reasoning steps
)

print("✓ Calculator agent ready!")

In [ ]:
# Test the agent with a multi-step calculation
query = "What is (15 * 4) + (100 / 5) - 3?"

print(f"Query: {query}\n")
print("Agent reasoning:")
print("=" * 60)

response = calculator_agent.chat(query)

print("\n" + "=" * 60)
print(f"Final Answer: {response}")

## 2. RAG Tool (Query Engine as Tool)

Agents can use query engines as tools for knowledge retrieval:

In [ ]:
# Load documents and create index
documents = SimpleDirectoryReader("../data/sample_docs").load_data()
index = VectorStoreIndex.from_documents(documents, show_progress=True)

# Create query engine
query_engine = index.as_query_engine(similarity_top_k=3)

print("\n✓ Index and query engine ready!")

In [ ]:
# Wrap query engine as a tool
knowledge_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="knowledge_base",
        description="Useful for answering questions about AI, machine learning, "
                    "Python programming, and related technical topics. "
                    "Use this tool when you need factual information from the knowledge base.",
    ),
)

print("✓ Knowledge tool created!")

In [ ]:
# Create agent with both calculation and knowledge tools
smart_agent = ReActAgent.from_tools(
    tools=[
        multiply_tool,
        add_tool,
        knowledge_tool,
    ],
    llm=Settings.llm,
    verbose=True,
)

print("✓ Smart agent ready!")

In [ ]:
# Test: Knowledge question
response = smart_agent.chat("What are the main types of machine learning?")
print(f"\nAnswer: {response}")

In [ ]:
# Test: Mixed question requiring both tools
response = smart_agent.chat(
    "If I have 3 types of machine learning and each type has 4 common algorithms, "
    "how many total algorithms are there? Also explain what supervised learning is."
)
print(f"\nAnswer: {response}")

## 3. Custom Tools with Complex Logic

Tools can contain arbitrary Python code:

In [ ]:
from datetime import datetime, timedelta
import json

def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def calculate_days_until(target_date: str) -> str:
    """Calculate the number of days until a target date (format: YYYY-MM-DD)."""
    try:
        target = datetime.strptime(target_date, "%Y-%m-%d")
        today = datetime.now()
        delta = target - today
        return f"{delta.days} days until {target_date}"
    except ValueError:
        return "Invalid date format. Please use YYYY-MM-DD"

def analyze_text(text: str) -> str:
    """Analyze text and return statistics like word count, character count, etc."""
    words = text.split()
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    stats = {
        "character_count": len(text),
        "word_count": len(words),
        "sentence_count": sentences,
        "average_word_length": sum(len(w) for w in words) / len(words) if words else 0,
    }
    
    return json.dumps(stats, indent=2)

# Create tools
time_tool = FunctionTool.from_defaults(fn=get_current_time)
days_until_tool = FunctionTool.from_defaults(fn=calculate_days_until)
analyze_tool = FunctionTool.from_defaults(fn=analyze_text)

print("✓ Utility tools created!")

In [ ]:
# Create utility agent
utility_agent = ReActAgent.from_tools(
    tools=[time_tool, days_until_tool, analyze_tool, knowledge_tool],
    llm=Settings.llm,
    verbose=True,
)

print("✓ Utility agent ready!")

In [ ]:
# Test utility functions
response = utility_agent.chat("What is the current time?")
print(f"\nAnswer: {response}")

In [ ]:
# Test text analysis
response = utility_agent.chat(
    "Analyze the following text and tell me about it: "
    "'Python is a versatile programming language. It is used for web development, "
    "data science, and artificial intelligence. Many developers love Python.'"
)
print(f"\nAnswer: {response}")

## 4. Agent with System Prompt

Customize agent behavior with system prompts:

In [ ]:
# System prompt for a research assistant
system_prompt = """
You are a research assistant specializing in AI and programming topics.

Guidelines:
1. Always search the knowledge base before answering factual questions
2. Provide structured, detailed responses
3. Cite sources when using information from the knowledge base
4. If you cannot find information, clearly state that
5. Use calculations tools when numerical analysis is needed

Be thorough but concise in your responses.
"""

research_agent = ReActAgent.from_tools(
    tools=[knowledge_tool, multiply_tool, add_tool],
    llm=Settings.llm,
    verbose=True,
    system_prompt=system_prompt,
)

print("✓ Research agent ready!")

In [ ]:
# Test research assistant behavior
response = research_agent.chat(
    "Research the topic of neural networks and provide a structured summary."
)
print(f"\nResearch Summary:\n{response}")

## 5. Agent Memory and Conversations

Agents maintain conversation history for multi-turn interactions:

In [ ]:
# Create conversational agent
conversational_agent = ReActAgent.from_tools(
    tools=[knowledge_tool, multiply_tool, add_tool],
    llm=Settings.llm,
    verbose=False,  # Less verbose for conversation
)

print("✓ Conversational agent ready!")
print("\n" + "="*60)
print("Starting conversation...")
print("="*60)

In [ ]:
# Multi-turn conversation
print("\nUser: What is machine learning?")
response1 = conversational_agent.chat("What is machine learning?")
print(f"Agent: {response1}")

In [ ]:
# Follow-up (agent remembers context)
print("\nUser: How is deep learning related to it?")
response2 = conversational_agent.chat("How is deep learning related to it?")
print(f"Agent: {response2}")

In [ ]:
# Another follow-up
print("\nUser: If I wanted to learn both, how many topics would that be?")
response3 = conversational_agent.chat(
    "If I wanted to learn both machine learning and deep learning, "
    "and each has about 5 main subtopics, how many topics total?"
)
print(f"Agent: {response3}")

In [ ]:
# View chat history
print("\n" + "="*60)
print("Chat History")
print("="*60)

for i, msg in enumerate(conversational_agent.chat_history):
    role = msg.role.value.upper()
    content = str(msg.content)[:100]
    print(f"\n{i+1}. {role}: {content}...")

In [ ]:
# Reset agent memory
conversational_agent.reset()
print("✓ Agent memory reset!")

## 6. OpenAI Function Calling Agent

For OpenAI models, you can use native function calling:

In [ ]:
from llama_index.agent.openai import OpenAIAgent

# Create OpenAI agent with function calling
openai_agent = OpenAIAgent.from_tools(
    tools=[knowledge_tool, multiply_tool, add_tool, time_tool],
    llm=OpenAI(model="gpt-4o-mini"),
    verbose=True,
)

print("✓ OpenAI function-calling agent ready!")

In [ ]:
# Test with function calling
response = openai_agent.chat(
    "What time is it now? Also, what is 25 * 4?"
)
print(f"\nAnswer: {response}")

## 7. Handling Agent Errors

Agents can encounter errors. Here's how to handle them gracefully:

In [ ]:
def risky_operation(value: int) -> str:
    """A function that might raise an error."""
    if value < 0:
        raise ValueError("Value must be non-negative")
    return f"Processed: {value * 2}"

risky_tool = FunctionTool.from_defaults(fn=risky_operation)

# Agent with error-prone tool
error_handling_agent = ReActAgent.from_tools(
    tools=[risky_tool],
    llm=Settings.llm,
    verbose=True,
    max_iterations=5,  # Limit iterations to prevent infinite loops
)

print("✓ Agent with error handling ready!")

In [ ]:
# Test with valid input
print("Test 1: Valid input")
response = error_handling_agent.chat("Process the value 10")
print(f"Response: {response}\n")

# Test with invalid input (agent should handle gracefully)
print("\nTest 2: Invalid input")
response = error_handling_agent.chat("Process the value -5")
print(f"Response: {response}")

## 8. Summary

You've learned how to build AI agents with LlamaIndex:

### Key Takeaways

| Concept | Description |
|---------|-------------|
| **FunctionTool** | Wrap Python functions as agent tools |
| **QueryEngineTool** | Use RAG as a tool |
| **ReActAgent** | Reasoning and acting pattern |
| **OpenAIAgent** | Native function calling |
| **System Prompt** | Customize agent behavior |
| **Memory** | Multi-turn conversations |

### Agent Design Patterns

1. **Tool Selection**: Provide clear tool descriptions
2. **Error Handling**: Set max iterations, handle exceptions
3. **Memory Management**: Reset when needed, use appropriate limits
4. **System Prompts**: Guide agent behavior and style

### Next Steps

In the next notebook, we'll explore Workflows for complex multi-step orchestration.

---

## Exercises

1. **Custom tool suite**: Create an agent with tools for your domain

2. **Multi-agent system**: Create multiple specialized agents

3. **Tool chaining**: Design tools that work together

4. **Evaluation**: Measure agent accuracy on a test set

In [ ]:
# Exercise space
# Build your own agent here!